## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from scipy.stats import randint

In [2]:
wine = datasets.load_wine()

x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)

clf = GradientBoostingClassifier(random_state=7)

clf.fit(x_train, y_train)

pred_y = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, pred_y)

print("Accuracy: ", acc)

Accuracy:  0.9555555555555556


# GridSearchCV

In [3]:
n_estimators = [10, 30, 50]
max_depth = [1, 2, 3]
param_grid = dict(n_estimators = n_estimators, max_depth = max_depth)

grid_search = GridSearchCV(clf, param_grid, scoring='accuracy',cv=5, n_jobs=-1, verbose=1)

grid_result = grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    1.2s finished
D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [4]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.962406 using {'max_depth': 1, 'n_estimators': 30}


In [5]:
clf_bestparam = GradientBoostingClassifier(n_estimators = grid_result.best_params_['n_estimators'], 
                                           max_depth = grid_result.best_params_['max_depth'])

clf_bestparam.fit(x_train, y_train)

pred_y = clf_bestparam.predict(x_test)

acc = metrics.accuracy_score(y_test, pred_y)

print("GridSearchCV accuracy: ", acc)

GridSearchCV accuracy:  0.9777777777777777


# RandomizedSearchCV

In [6]:
param_dist = {"max_depth": [3, 5, 7],
              "max_features": randint(1, 11),
              "min_samples_split": randint(2, 6),
              "learning_rate": [1, 0.1, 0.01, 0.001, 0.5, 0.05, 0.005]}

n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)

random_result = random_search.fit(x_train, y_train)

In [7]:
print("Best Accuracy: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best Accuracy: 0.970085 using {'learning_rate': 0.05, 'max_depth': 3, 'max_features': 5, 'min_samples_split': 5}


In [8]:
clf_random_bestparam = GradientBoostingClassifier(learning_rate = random_result.best_params_['learning_rate'], 
                                                  max_depth = random_result.best_params_['max_depth'],
                                                 max_features = random_result.best_params_['max_features'],
                                                 min_samples_split = random_result.best_params_['min_samples_split'])

clf_random_bestparam.fit(x_train, y_train)

pred_y = clf_random_bestparam.predict(x_test)

acc = metrics.accuracy_score(y_test, pred_y)

print("GridSearchCV accuracy: ", acc)

GridSearchCV accuracy:  1.0
